In [1]:
import cv2
import random
import numpy as np
import pandas as pd
import math

In [2]:
# Lectura de dataframe/parquet con datos de todos los videos
dataframe = pd.read_parquet('../01_datos/00_raw/raw_parquet')

""" 
    ARMADO DE COLUMNAS PARA LECTURA DE DATAFRAME 
    Columnas numericas del 0 a 1085 representando la cantidad de puntos totales.
"""
columnsToRead = [str(i) for i in range(1086)]
columnsToRead.append('sign')

#### A TENER EN CUENTA:

<span style="color:red">
CAMBIAR EL ORDEN DE LAS PALABRAS EN ALGUNA DE LAS 2 LISTAS VA A LLEVAR A UNA LECTURA INCORRECTA DE LOS DATOS 

EN EL PARQUET COMPLETO ORIGINAL SI SE REALIZA UN CAMBIO (en el orden en que se almacenaron las palabras) ESTE DEBE REALIZARSE SOBRE LAS DEMAS NOTEBOOKs DE

LECTURA Y CREACION DE PARQUETS YA SEA SOBRE EL ORIGINAL O LOS DE MUESTREO
</span>

In [3]:
SIGNS_LIST = ['nacer','comida','brillante','mujer', 'hijo', 'hombre', 'lejos', 'aprender', 'espumadera','amargo','leche','Uruguay','pais','donde','ninguno','nombre','perfume','sordo','comprar','encontrar', 'nave espacial']

# se toma la lista original de señas ya que estas en el orden en que se encuentran, su indice representa correctamente el nombre del video original
FULL_SIGNS_LIST = ['opaco', 'rojo', 'verde', 'amarillo', 'brillante', 'celeste', 'colores', 'rosa', 'mujer', 'enemigo', 'hijo', 'hombre', 'lejos','cajón','nacer','aprender','llamar','espumadera','amargo','dulce','leche','agua','comida','Argentina','Uruguay','pais','donde','apellido','burla','cumpleanos','desayuno','foto','hambre','mapa','moneda','musica','nave espacial','ninguno','nombre','paciencia','perfume','sordo','trampa','arroz','asado','caramelo','chicle','fideos','yogurt','aceptar','agradecer','apagar','aparecer','aterrizar','atrapar','ayudar','bailar','bañarse','comprar','copiar','correr','darse cuenta','dar','encontrar']

NUMBER_OF_PERSONS = 10
NUMBER_OF_VIDEOS_PER_PERSON = 5
AMOUNT_OF_FRAMES = 10
# semilla para la generacion de numero aleatorios en la eleccion de frames
RANDOM_SEED = 48

##### ACTIVACION/DESACTIVACION DE CARA y CANTIDAD DE FRAMES
La variable "useFacePoints" añadirá las columnas necesarias para los puntos faciales que serán añadidos en el parquet a construir.
La variable "amount_of_frames" aumentará las cantidad de columnas necesarias, en este caso, cuantos frames se tomaran de cada video para construir el parquet.


In [5]:
frame_columns = []

FACE_POINTS = 468
POSE_POINT_INDEXES = [k for k in range(23)] #de los 33 puntos solo tomamos los hombros, brazos, cabeza (SIN CINTURA debido a los videos del set de datos)
LEFT_HAND_POINTS = 21
RIGHT_HAND_POINTS = 21

# CAMBIAR ESTE BOOLEANO PARA TOMAR PUNTOS FACIALES O NO (se tomarán CEJAS y BOCA)
USE_FACE_POINTS = True

# https://github.com/tensorflow/tfjs-models/commit/838611c02f51159afdd77469ce67f0e26b7bbb23#diff-e5d31503f11c6bae62542ea89982152514b81906dff0b718e44708bcf22aa361
# https://github.com/ManuelTS/augmentedFaceMeshIndices/blob/master/Left_Eye.jpg
# https://github.com/google/mediapipe/blob/a908d668c730da128dfa8d9f6bd25d519d006692/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png

# No he repetido los puntos que tenian en comun ciertos arreglos, como los puntos faciales en los datos
# obtenidos de los videos se encuentran primeros, estos indices se corresponden

rightEyebrowUpper = [156, 70, 63, 105, 66, 107, 55, 193]
rightEyebrowLower = [35, 124, 46, 53, 52, 65]

leftEyebrowUpper  = [383, 300, 293, 334, 296, 336, 285, 417]
leftEyebrowLower  = [265, 353, 276, 283, 282, 295]
    
lipsUpperOuter    = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter    = [146, 91, 181, 84, 17, 314, 405, 321, 375]
lipsUpperInner    = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner    = [95, 88, 178, 87, 14, 317, 402, 318, 324]

facePointsIndexes = rightEyebrowUpper + rightEyebrowLower + leftEyebrowUpper + leftEyebrowLower + lipsUpperOuter + lipsLowerOuter + lipsUpperInner + lipsLowerInner

# print("Puntos de cara "+str(len(facePointsIndexes)))
# print("Puntos de Posicion "+str(len(POSE_POINT_INDEXES)))

def build_columns():
    print('Frames: ' + str(AMOUNT_OF_FRAMES))
    print('Usar puntos faciales: ' + str(USE_FACE_POINTS))
    global frame_columns
    frame_columns = []

    for frame in range(AMOUNT_OF_FRAMES):
        
        if (USE_FACE_POINTS):
            for index in range (len(facePointsIndexes)):
                frame_columns.append(f'''fr_{frame}_face_p{index}_x''')
                frame_columns.append(f'''fr_{frame}_face_p{index}_y''')
        
        for index in range (len(POSE_POINT_INDEXES)):
            frame_columns.append(f'''fr_{frame}_pose_p{index}_x''')
            frame_columns.append(f'''fr_{frame}_pose_p{index}_y''')

        for index in range (LEFT_HAND_POINTS):
            frame_columns.append(f'''fr_{frame}_left_hand_p{index}_x''')
            frame_columns.append(f'''fr_{frame}_left_hand_p{index}_y''')

        for index in range (RIGHT_HAND_POINTS):
            frame_columns.append(f'''fr_{frame}_right_hand_p{index}_x''')
            frame_columns.append(f'''fr_{frame}_right_hand_p{index}_y''')
    frame_columns.append("sign")
    print('Cantidad de columnas: '+ str(len(frame_columns)))


In [6]:
def values_validation(frame):
    for i in range(len(frame)):
        if (frame[i] > 1):
            if (frame[i] > 2.8): 
                print("Valor que supera al 2.8: "+ str(frame[i]))
            frame[i] = 1
    
def getVideoKeypoints(sign_iteration, person, video):
    frames = []

    ''' El valor 201 corresponde a la longitud fija que se otorgó a todos los videos
    ya sea que estos la cumplan o no. Al momento de armar el parquet 
    con todos los videos procesados, aquellos que no llegaban a esta longitud se
    completaron con "frames" de valores 3 (valor que no debería existir en los datos 
    originales por la normalizacion que realiza MediaPipe)
    El valor 50 proviene de la cantidad de videos para cada seña
    
    Bloque de Seña: la seña me indica el bloque, y como cada bloque está compuesto por 50videos por 201 filas cada 1, el salto de bloque me lo da el indice de la seña que se está procesando comenzando desde el 0.
    Bloque de seña: 50*201*sign_index

    Bloque de persona: cada persona filma 5 videos por seña y como tenemos 10 personas, para acceder a la correspondiente realizamos, 5 videos de la persona * 201 frames por cada video
    Bloque de persona: indiceDePersona * 5 * 201

    Bloque de video: Una vez que conocemos el Bloque de Seña y Bloque de persona queda posicionar el video. Cada video esta compuesto por 201 frames
    Bloque de video: 201*indiceDeVideo
    '''
    for frameIndex in range(201):
        frame = []
        frame = dataframe.iloc[ (50*201*sign_iteration)+(person*5*201)+(201*video) + frameIndex ].to_list()
        # quito ultima columna con la seña de cada frame/fila
        frame.pop()

        frames.append(frame)
    return frames

def store_dataset(dataset, name):
    dataset.to_parquet('./01_criterios/{0}.parquet'.format(name))
    print('Se almacenó: ./01_criterios/{0}.parquet'.format(name))

# MUESTREO PODA
Se realiza una poda de los frames iniciales y finales de cada video. Para ello se calculará el 15% de la cantidad de frames para cada video y este valor (cantidad de frames) no serán tenidas en cuenta. Por ejemplo, si el valor es de 10, no se tomarán en cuenta los primeros 10 y ultimos 10 frames (como si estos hubieran sido podados). Luego se tomará de forma aleatoria los frames restantes para el aprendizaje.

In [15]:
def build_dataset():
    dataFrameRows = []

    # sign_iteration se utiliza para poder acceder a los bloques de la seña correspondiente
    # por ejemplo la seña 'Brillante' se encontrará primera en el parquet pero con la lista de señas no es posible conocer su posicion en el parquet
    # debido a esto se utiliza esta variable
    sign_iteration = 0

    random.seed(RANDOM_SEED)

    for sign in range(len(FULL_SIGNS_LIST)):
        if (SIGNS_LIST.count(FULL_SIGNS_LIST[sign])):
            for j in range(10):
                for k in range(5):
                    cap = cv2.VideoCapture(f'''C:/Users/facur/Desktop/tesis_LSA/codigos_datos_tesis/LSA64/all_cut/0{str(sign+1).zfill(2)}_0{str(j+1).zfill(2)}_00{k+1}.mp4''')
                    framesLength = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                    fifth_percent = int(framesLength*0.15)

                    video_keypoints = getVideoKeypoints(sign_iteration = sign_iteration, person = j, video = k)

                    randomFrames = []
                    
                    # si podando tenemos frames para completar todo, es decir,
                    # luego de la poda la cantidad de frames es mayor o igual a la cantidad de frames con la que queremos trabajar <AMOUNT_OF_FRAMES> 
                    # El factor *2 se realiza ya que la poda es del inicio y del final
                    if (framesLength - (fifth_percent*2) > AMOUNT_OF_FRAMES): 
                        for i in range(AMOUNT_OF_FRAMES):
                            number = random.randint(fifth_percent, framesLength - fifth_percent)
                            while(randomFrames.count(number)):
                                number = random.randint(fifth_percent, framesLength - fifth_percent)
                            randomFrames.append(number)
                    else:
                        # como el video con menor cantidad de frames es 14, este bloque "poda" como minimo los primeros 2 y ultimos 2 frames. 
                        # Y poda con valores superiores en aquellos casos donde la cantidad de frames luego de la poda inicial y final no lleguen a la cantidad de <AMOUNT_OF_FRAMES> frames
                        frames_to_jump = math.floor((framesLength - AMOUNT_OF_FRAMES) / 2)
                        for i in range(AMOUNT_OF_FRAMES):
                            number = random.randint(frames_to_jump, framesLength - frames_to_jump)
                            while(randomFrames.count(number)):
                                number = random.randint(frames_to_jump, framesLength - frames_to_jump)
                            randomFrames.append(number)
                        
                    randomFrames.sort()

                    if (framesLength == 14):
                        print(randomFrames)
                        print(fifth_percent)
                    
                    aux_array = []
                    for randomFrame in randomFrames:
                        values_validation(video_keypoints[randomFrame])
                        if (USE_FACE_POINTS):
                            for faceKeypoint in facePointsIndexes:
                                aux_array.append(video_keypoints[randomFrame][faceKeypoint*2])     # X
                                aux_array.append(video_keypoints[randomFrame][faceKeypoint*2+1])   # Y
                        # POSE
                        for poseKeypoint in range(936, 936+len(POSE_POINT_INDEXES)*2):
                            aux_array.append(video_keypoints[randomFrame][poseKeypoint])
                        for keypoint in range(1002, 1086):
                            aux_array.append(video_keypoints[randomFrame][keypoint])
                    aux_array.append(FULL_SIGNS_LIST[sign])

                    dataFrameRows.append(aux_array)
            sign_iteration = sign_iteration + 1
                
    return pd.DataFrame(dataFrameRows,columns=frame_columns)

In [13]:
def build_extended_dataset():
    dataFrameRows = []

    # sign_iteration se utiliza para poder acceder a los bloques de la seña correspondiente
    # por ejemplo la seña 'Brillante' se encontrará primera en el parquet pero con la lista de señas no es posible conocer su posicion en el parquet
    # debido a esto se utiliza esta variable
    sign_iteration = 0

    random.seed(RANDOM_SEED)

    for sign in range(len(FULL_SIGNS_LIST)):
        if (SIGNS_LIST.count(FULL_SIGNS_LIST[sign])):
            for j in range(10):
                for k in range(5):
                    cap = cv2.VideoCapture(f'''C:/Users/facur/Desktop/tesis_LSA/codigos_datos_tesis/LSA64/all_cut/0{str(sign+1).zfill(2)}_0{str(j+1).zfill(2)}_00{k+1}.mp4''')
                    framesLength = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                    fifth_percent = int(framesLength*0.15)

                    video_keypoints = getVideoKeypoints(sign_iteration = sign_iteration, person = j, video = k)

                    randomFrames = []
                
                    if (framesLength - (fifth_percent*2) > AMOUNT_OF_FRAMES): #si podando tenemos frames para completar todo, *2 ya que es poda del inicio y del final
                        for i in range(AMOUNT_OF_FRAMES):
                            number = random.randint(fifth_percent, framesLength - fifth_percent)
                            while(randomFrames.count(number)):
                                number = random.randint(fifth_percent, framesLength - fifth_percent)
                            randomFrames.append(number)
                    else:
                        # repito frames aleatorios para completar la cantidad requerida de frames
                        for number in range(AMOUNT_OF_FRAMES):
                            # se itera desde 0 hasta la cantidad de Frames Objetivo que tenemos, por ejemplo queremos armar un set de datos
                            # con 30 frames, pero el video cuenta con 14 frames, en ese caso, se itera desde 0 a 30 calculando el modulo
                            # entre la iteracion actual y la cantidad de frames del video (14), y así se evita acceder a frames inexistentes
                            randomFrames.append(number % framesLength)
                
                    randomFrames.sort()
                    
                    aux_array = []
                    for randomFrame in randomFrames:
                        values_validation(video_keypoints[randomFrame])
                        if (USE_FACE_POINTS):
                            for faceKeypoint in facePointsIndexes:
                                aux_array.append(video_keypoints[randomFrame][faceKeypoint*2])     # X
                                aux_array.append(video_keypoints[randomFrame][faceKeypoint*2+1])   # Y
                        # POSE
                        for poseKeypoint in range(936, 936+len(POSE_POINT_INDEXES)*2):
                            aux_array.append(video_keypoints[randomFrame][poseKeypoint])
                        for keypoint in range(1002, 1086):
                            aux_array.append(video_keypoints[randomFrame][keypoint])
                    aux_array.append(FULL_SIGNS_LIST[sign])

                    dataFrameRows.append(aux_array)
            sign_iteration = sign_iteration + 1
                
    return pd.DataFrame(dataFrameRows,columns=frame_columns)

SE PROCEDE A CONSTRUIR LOS DISTINTOS DATASETS

In [23]:
# -------------- DATASET muestreo 10 frames SIN cara
AMOUNT_OF_FRAMES = 10
USE_FACE_POINTS = False

build_columns()
dataset = build_dataset()
print('Nro de Filas: '+str(len(dataset.axes[0])))
print('Nro de Columnas: '+str(len(dataset.axes[1])))
# print(dataset.head())
store_dataset(dataset, 'criterio_poda_10fr_no_face')

# -------------- DATASET muestreo 10 frames CON cara
AMOUNT_OF_FRAMES = 10
USE_FACE_POINTS = True

build_columns()
dataset = build_dataset()
print('Nro de Filas: '+str(len(dataset.axes[0])))
print('Nro de Columnas: '+str(len(dataset.axes[1])))
# print(dataset.head())
store_dataset(dataset, 'criterio_poda_10fr_with_face')

# -------------- DATASET muestreo 30 frames SIN cara
AMOUNT_OF_FRAMES = 30
USE_FACE_POINTS = False

build_columns()
dataset = build_extended_dataset()
print('Nro de Filas: '+str(len(dataset.axes[0])))
print('Nro de Columnas: '+str(len(dataset.axes[1])))
# print(dataset.head())
store_dataset(dataset, 'criterio_poda_30fr_no_face')

# -------------- DATASET muestreo 30 frames CON cara
AMOUNT_OF_FRAMES = 30
USE_FACE_POINTS = True

build_columns()
dataset = build_extended_dataset()
print('Nro de Filas: '+str(len(dataset.axes[0])))
print('Nro de Columnas: '+str(len(dataset.axes[1])))
# print(dataset.head())
store_dataset(dataset, 'criterio_poda_30fr_with_face')

del dataset

Frames: 10
Usar puntos faciales: False
Cantidad de columnas: 1301
[2, 3, 4, 5, 7, 8, 9, 10, 11, 12]
2
Nro de Filas: 1050
Nro de Columnas: 1301
Se almacenó: ./01_criterios/criterio_random_10fr_no_face.parquet
Frames: 10
Usar puntos faciales: True
Cantidad de columnas: 2661
[2, 3, 4, 5, 7, 8, 9, 10, 11, 12]
2
Nro de Filas: 1050
Nro de Columnas: 2661
Se almacenó: ./01_criterios/criterio_random_10fr_with_face.parquet
Frames: 30
Usar puntos faciales: False
Cantidad de columnas: 3901
Nro de Filas: 1050
Nro de Columnas: 3901
Se almacenó: ./01_criterios/criterio_random_30fr_no_face.parquet
Frames: 30
Usar puntos faciales: True
Cantidad de columnas: 7981
Nro de Filas: 1050
Nro de Columnas: 7981
Se almacenó: ./01_criterios/criterio_random_30fr_with_face.parquet
